# **Launch Sites Locations Analysis with Folium**

**Note:** *This notebook includes interactive Folium maps that require a live Jupyter environment to render properly. If you're viewing this on GitHub, the maps will not display inline. Please click the link under each map cell to open it in a separate tab. For full interactivity, launch the notebook on Binder using the badge in the README file of this repository.*

---
The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis sections, we have visualized the SpaceX launch dataset and discovered some preliminary correlations between the launch site and success rates. In this part, we will be performing more interactive visual analytics using `Folium`.


## Objectives



- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

After achieving the above tasks, we will be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import wget
import pandas as pd
import os

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Marking all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
# Downloading and read the `spacex_launch_geo.csv`
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
save_dir = './Datasets'
file_name = 'spacex_launch_geo.csv'
os.makedirs(save_dir,exist_ok=True)
file_path=os.path.join(save_dir,file_name)
wget.download(url, out=file_path)
print(f'\nFile saved to: {file_path}')
spacex_df=pd.read_csv('./Datasets/spacex_launch_geo.csv')
spacex_df.head()

100% [................................................................................] 7710 / 7710
File saved to: ./Datasets\spacex_launch_geo.csv


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site.


In [4]:
# Selectting relevant sub-columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [6]:
# Creating a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Creating a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Creating an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)
site_map.save('./assets/map1.html')
site_map

and we will find a small yellow circle near the city of Houston. We can zoom-in to see a larger circle [**CLICK HERE TO SEE THE MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map1.html).

Now, let's add a circle for each launch site in data frame `launch_sites`


In [7]:
# Initiallizing the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

coor1=[28.562302,-80.577356]
coor2=[28.563197,-80.576820]
coor3=[28.573255,-80.646895]
coor4=[34.632834,-120.610745]

# For each launch site, we will add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
circle1 = folium.Circle(coor1, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
marker1 = folium.map.Marker(
    coor1,
    # Creating an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',
        )
    )
circle2 = folium.Circle(coor2, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40'))
marker2 = folium.map.Marker(
    coor2,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',
        )
    )
circle3 = folium.Circle(coor3, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('KSC LC-39A'))
marker3 = folium.map.Marker(
    coor3,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A',
        )
    )
circle4 = folium.Circle(coor4, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('VAFB SLC-4E'))
marker4 = folium.map.Marker(
    coor4,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
        )
    )
site_map.add_child(circle1)
site_map.add_child(circle2)
site_map.add_child(circle3)
site_map.add_child(circle4)
site_map.add_child(marker1)
site_map.add_child(marker2)
site_map.add_child(marker3)
site_map.add_child(marker4)
site_map.save('./assets/map2.html')
site_map

All launch sites are near the Equator, which helps with fuel efficiency due to Earth’s rotation. They’re also close to the coast, allowing safe launches over water and easier booster recovery [**CLICK HERE TO SEE THE MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map2.html).

## Marking the success/failed launches for each site on the map

Next, let's enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not:

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()

Then we will create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [10]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Define a function to assign color
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

# Apply the function and create the new column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [11]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in `spacex_df` data frame, we will add a `folium.Marker` to `marker_cluster`


In [12]:
site_map.add_child(marker_cluster)

# Loop through each launch record
for index, record in spacex_df.iterrows():
    # Extract coordinates
    lat = record['Lat']
    lon = record['Long']
    
    # Creating a folium Marker with color-coded icon
    marker = folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Adding marker to the cluster
    marker_cluster.add_child(marker)
    
site_map.save('./assets/map3.html')
site_map

From the color-labeled markers in marker clusters, we are able to easily identify which launch sites have relatively high success rates [**CLICK HERE TO SEE THE MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map3.html).


## Calculating the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway) [**CLICK HERE TO SEE THE MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map4.html).

In [13]:
# Adding Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map.save('./assets/map4.html')
site_map

Now we will zoom in to a launch site and explore its proximity to see if we can easily find any railway, highway, coastline, etc.

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

We will mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site [**CLICK HERE TO SEE THE MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map5.html).


In [15]:
# finding coordinate of the closet coastline
distance_coastline = calculate_distance(28.562302, -80.577356, 28.56266, -80.56789)

In [16]:
# Creating and adding a folium.Marker on the selected closest coastline point on the map
# Displaying the distance between coastline point and launch site using the icon property 
cost_coor=[28.56266, -80.56789]
distance_marker = folium.Marker(
    location=cost_coor,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

In [17]:
# Creating a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[coor1,cost_coor], weight=1)
site_map.add_child(lines)
site_map.save('./assets/map5.html')
site_map

Similarly, we will draw a line betwee a launch site to its closest city, railway, highway, etc. [**CLICK HERE TO SEE THE FINAL MAP**](https://raw.githack.com/filipe-braiman/spacex-falcon9-landing-prediction/master/assets/map_final.html).


In [18]:
# Creating a marker with distance to a closest city, railway, highway, etc.
# Drawing a line between the marker to the launch site

distance_way = calculate_distance(28.573255,	-80.646895, 28.56395, -80.58684)

city_coor=[28.56395, -80.58684]
distance_marker = folium.Marker(
    location=city_coor,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_way),
        )
    )
site_map.add_child(distance_marker)

lines=folium.PolyLine(locations=[coor2,city_coor], weight=1)
site_map.add_child(lines)

In [19]:
distance_city = calculate_distance(28.562302, -80.577356, 28.61225, -80.81124)

city_coor=[28.61225, -80.81124]
distance_marker = folium.Marker(
    location=city_coor,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_marker)

lines=folium.PolyLine(locations=[coor3,city_coor], weight=1)
site_map.add_child(lines)

In [20]:
distance_rway = calculate_distance(34.632834, -120.610745, 34.63487, -120.62431)

rway_coor=[34.63487, -120.62431]
distance_marker = folium.Marker(
    location=rway_coor,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rway),
        )
    )
site_map.add_child(distance_marker)

lines=folium.PolyLine(locations=[coor4,rway_coor], weight=1)
site_map.add_child(lines)
site_map.save('./assets/map_final.html')
site_map

We can conclude that launch sites are located near railways, highways, and coastlines, which supports efficient transport and safe launch paths. They also maintain a buffer from major cities, reducing risk and ensuring operational safety.

## Next Steps:

Now we have discovered many interesting insights related to the launch sites' location, in a very interactive way. Next, we will build a dashboard using Ploty Dash on detailed launch records.

## Author
**Filipe B. Carvalho**

**Email:** [filipebraiman@gmail.com](mailto:filipebraiman@gmail.com)  
**LinkedIn:** [linkedin.com/in/filipe-b-carvalho](https://www.linkedin.com/in/filipe-b-carvalho)  
**GitHub:** [github.com/filipe-braiman](https://github.com/filipe-braiman)  

### About Me  
Data and AI professional with experience in **AI model evaluation, data annotation, and NLP projects**, currently contributing to AI initiatives at **Huawei**. Skilled in **Python, SQL, data visualization, machine learning, AI, and dataset building**, and certified through the **IBM Data Science Professional Certificate**. Multilingual in **Portuguese, English, Spanish, Turkish, and French**, bringing a linguistic and analytical perspective to data-driven problem solving. Passionate about leveraging data and AI to create practical, high-impact solutions.

---

### Version History

| Version | Date       | Changes                         |
|:--------:|:-----------|:--------------------------------|
| 1.0      | 2025-10-29 | First publication of the notebook |